In [1]:
from agm_te.dataset import DataSet
from agm_te.estimate import agm_estimate_TE

In [2]:
# Set up the compute device
import torch; torch.set_printoptions(sci_mode=None)
if torch.cuda.is_available(): # Check if CUDA is available
    compute_device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    compute_device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


### Initialize the Simulator (from the `te_datasim` package) and generate time series for variables $X$ and $Y$:

In [5]:
from te_datasim.lineargaussian import BVLinearGaussianSimulator

bivar = BVLinearGaussianSimulator()
print("Analytical TE X -> Y: ", bivar.analytic_transfer_entropy('X', 'Y'))
print("Analytical TE Y -> X: ", bivar.analytic_transfer_entropy('Y', 'X'), "\n")

bivar_data_dict = {'X':[], 'Y':[]}
for i in range(10):
    X, Y = bivar.simulate(2000, seed=i)
    bivar_data_dict['X'].append(X)
    bivar_data_dict['Y'].append(Y)

bivar_data = DataSet(bivar_data_dict)
print(bivar_data)

Analytical TE X -> Y:  0.0
Analytical TE Y -> X:  0.1276 

DataSet of 20000 timesteps across 10 trajectories for 2 variables:
	 X is 1 dimensional
	 Y is 1 dimensional



### Set up the parameters for the AGMs:

In [4]:
model_params = {
    'obs_model_type':   'Gaussian', # Gaussian observation model
    'dyn_model_type':   'MLPTanh',  # Multi-layer perceptron with tanh activation dynamics model, 2 layers of 16 hidden units
    'hidden_size':      16,
    'num_layers':       2,
    'compute_device':   compute_device # Use the compute device we set up (ideally GPU)
}

train_params = {
    'batch_size':       1,
    'epochs':           1000,
    'optimize':         'sgd',
    'learning_rate':    0.01,
}

### Estimate $\mathcal{T}_{X \to Y}$

In [5]:
model_1, model_2, te = agm_estimate_TE(bivar_data, model_params, train_params, var_from='X', var_to='Y')
print("Estimated TE X -> Y: ", te, "                        ")

Estimated TE X -> Y:  -0.0                         


### Estimate $\mathcal{T}_{Y \to X}$

In [6]:
model_1, model_2, te = agm_estimate_TE(bivar_data, model_params, train_params, var_from='Y', var_to='X')
print("Estimated TE Y -> X: ", te, "                        ")

Estimated TE Y -> X:  0.1266                         
